In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
"""Trains a ResNet on the CIFAR10 dataset.

ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf

ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf
"""

import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import Callback

import numpy as np
import pandas as pd
from datetime import datetime
import time
import os
from keras.constraints import Constraint
from keras import initializers, layers
from keras.layers import Lambda

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
depth = n * 6 + 2 # n=3 --> 20, n=5 --> 32, n=7 --> 44, n=9 --> 56

# Model name, depth and version
model_type = 'ResNet%dv%d_noshare' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


class ShrinkageConstraint(Constraint):
    def __init__(self, axis=0):
        self.axis = axis    
    
    def __call__(self, w):
        # apply unitnorm
        #w = w / (K.epsilon() + K.sqrt(K.sum(K.square(w),
        #                                    axis=self.axis,
        #                                    keepdims=True)))
        
        # apply non negative
        w *= K.cast(K.greater_equal(w, 0.), K.floatx())
        # apply max value to be 1
        w *= K.cast(K.less_equal(w, 1.), K.floatx())
        return w


class ShrinkageFactor(layers.Layer):
    """
    This is the sigma object in the algorithm 1 by Beygelzimer (Online Gradient Boosting)
    """
    def __init__(self, step_size, trainable=True, **kwargs):
        self.step_size = step_size
        self.trainable = trainable
        super(ShrinkageFactor, self).__init__(**kwargs)
    
    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name='highway', 
                                 shape=(1, 1),
                                 initializer=initializers.Zeros(),
                                 constraint=ShrinkageConstraint(),
                                 regularizer=l2(0.01),
                                 trainable=self.trainable)
        self.count = K.variable(0, name="epoch")
        super(ShrinkageFactor, self).build(input_shape)  # Be sure to call this at the end
    
    def call(self, x):
        return (1-self.step_size*self.W)*x
        updates = []
        
        if self.count < 80:
            updates.append((self.count, self.count+1))
            return x
        else:
            updates.append((self.count, self.count+1))
            return (1-self.step_size*self.W)*x
    
    def compute_output_shape(self, input_shape):
        if isinstance(input_shape, list):
            return input_shape[0]
        return input_shape

class TimingCallback(Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    
    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
    
    def on_epoch_end(self, batch, logs={}):
        # write stuff to disc here...
        self.times.append(time.time() - self.epoch_time_start)

def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True,
                 stack=0, 
                 res_block="placeholder"):
    """2D Convolution-Batch Normalization-Activation stack builder
    
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
        
        stack (int): stack number for layer naming purposes
        res_block (string): name of the res_block for naming purposes
    
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4),
                  name=f"resnet_{stack}_{res_block}"
                  )
    
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization(name=f"bn_{stack}_{res_block}")(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization(f"bn_{stack}_{res_block}")(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_block(x, stack, res_block, num_filters, boost=True):
    strides = 1
    if stack > 0 and res_block == 0:  # first layer but not first stack
        strides = 2  # downsample
    y = resnet_layer(inputs=x,
                     num_filters=num_filters,
                     strides=strides,
                     stack=stack,
                     res_block=f"{res_block}a")
    y = resnet_layer(inputs=y,
                     num_filters=num_filters,
                     activation=None,
                     stack=stack,
                     res_block=f"{res_block}b")
    if stack > 0 and res_block == 0:  # first layer but not first stack
        # linear projection residual shortcut connection to match
        # changed dims
        x = resnet_layer(inputs=x,
                         num_filters=num_filters,
                         kernel_size=1,
                         strides=strides,
                         activation=None,
                         batch_normalization=False,
                         stack=stack,
                         res_block=f"{res_block}c")
    
    if boost:
        step_size = 1.0
        y = ShrinkageFactor(step_size, False, name=f"shrinkage_{stack}_{res_block}")(y)
        # x = Lambda(lambda x: x * step_size, name=f"shrinkage_lambda_{stack}_{res_block}")(x)
    
    x = keras.layers.add([x, y], name=f"add_{stack}_{res_block}")
    x = Activation('relu')(x)
    return x


Using TensorFlow backend.


170500096/170498071 [==============================] - 41s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [3]:
# model = resnet_v1(input_shape=input_shape, depth=depth)
# we shall hardcode the model...with num_res_blocks=3


# model = resnet_v1(input_shape=input_shape, depth=depth)
# we shall hardcode the model...with num_res_blocks=3

num_filters = 16

inputs = Input(shape=input_shape)
x = resnet_layer(inputs=inputs, res_block='preprocessing')

block0_0 = resnet_block(x, 0, 0, num_filters)
block0_1 = resnet_block(block0_0, 0, 1, num_filters)
block0_2 = resnet_block(block0_1, 0, 2, num_filters)

block1_0 = resnet_block(block0_2, 1, 0, num_filters*2)
block1_1 = resnet_block(block1_0, 1, 1, num_filters*2)
block1_2 = resnet_block(block1_1, 1, 2, num_filters*2)

block2_0 = resnet_block(block1_2, 2, 0, num_filters*4)
block2_1 = resnet_block(block2_0, 2, 1, num_filters*4)
block2_2 = resnet_block(block2_1, 2, 2, num_filters*4)

block_output = AveragePooling2D(pool_size=8, name="avg_pool_2_2")(block2_2)
block_output_flatten = Flatten(name="flatten_2_2")(block_output)
#y = Dense(128)(block_output_flatten)

pred_layer_0 = Dense(num_classes,
                activation='softmax',
                name='pred_layer_0')

pred_layer_1 = Dense(num_classes,
                activation='softmax',
                name='pred_layer_1')

pred_layer_2 = Dense(num_classes,
                activation='softmax',
                name='pred_layer_2')
outputs = pred_layer_2(block_output_flatten)


model = Model(inputs=[inputs], outputs=[outputs])
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

# print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
save_dir = "/content/gdrive/My Drive/colab/weights/"
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

time_cb = TimingCallback()

callbacks = [checkpoint, lr_reducer, lr_scheduler, time_cb]

print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    # set input mean to 0 over the dataset
    featurewise_center=False,
    # set each sample mean to 0
    samplewise_center=False,
    # divide inputs by std of dataset
    featurewise_std_normalization=False,
    # divide each input by its std
    samplewise_std_normalization=False,
    # apply ZCA whitening
    zca_whitening=False,
    # epsilon for ZCA whitening
    zca_epsilon=1e-06,
    # randomly rotate images in the range (deg 0 to 180)
    rotation_range=0,
    # randomly shift images horizontally
    width_shift_range=0.1,
    # randomly shift images vertically
    height_shift_range=0.1,
    # set range for random shear
    shear_range=0.,
    # set range for random zoom
    zoom_range=0.,
    # set range for random channel shifts
    channel_shift_range=0.,
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    # value used for fill_mode = "constant"
    cval=0.,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False,
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0)

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)




Learning rate:  0.001
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
resnet_0_preprocessing (Conv2D) (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
bn_0_preprocessing (BatchNormal (None, 32, 32, 16)   64          resnet_0_preprocessing[0][0]     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           bn_0_preprocessing[0][0]         
_______________________________________________________________________________________

In [4]:


# Fit the model on the batches generated by datagen.flow().
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    validation_data=(x_test, [y_test]),
                    steps_per_epoch=1600,
                    epochs=epochs, verbose=2, workers=10,
                    callbacks=callbacks)
hist_df = pd.DataFrame(hist.history)
hist_df['times'] = time_cb.times[-hist_df.shape[0]:]
hist_df.to_csv('/content/gdrive/My Drive/colab/weights/training_history_resnetv1_noshare{}.csv'.format(datetime.now().strftime('%Y-%m-%d_%H-%M-%S')), index=True)

# Score trained model.
scores = model.evaluate(x_test, [y_test], verbose=1)
print('Test output:', scores)


Epoch 1/200
Learning rate:  0.001
 - 120s - loss: 1.5180 - acc: 0.4937 - val_loss: 1.4964 - val_acc: 0.5172

Epoch 00001: val_acc improved from -inf to 0.51720, saving model to /content/gdrive/My Drive/colab/weights/cifar10_ResNet20v1_noshare_model.001.h5
Epoch 2/200
Learning rate:  0.001
 - 113s - loss: 1.1345 - acc: 0.6460 - val_loss: 1.2237 - val_acc: 0.6345

Epoch 00002: val_acc improved from 0.51720 to 0.63450, saving model to /content/gdrive/My Drive/colab/weights/cifar10_ResNet20v1_noshare_model.002.h5
Epoch 3/200
Learning rate:  0.001
 - 114s - loss: 0.9812 - acc: 0.7075 - val_loss: 1.1336 - val_acc: 0.6592

Epoch 00003: val_acc improved from 0.63450 to 0.65920, saving model to /content/gdrive/My Drive/colab/weights/cifar10_ResNet20v1_noshare_model.003.h5
Epoch 4/200
Learning rate:  0.001
 - 113s - loss: 0.8926 - acc: 0.7420 - val_loss: 1.0386 - val_acc: 0.7084

Epoch 00004: val_acc improved from 0.65920 to 0.70840, saving model to /content/gdrive/My Drive/colab/weights/cifar10

In [0]:
model.save_weights('/content/gdrive/My Drive/colab/weights/cifar10_noshare_boost_oct16.h5')

Done

In [6]:
1+2

3